## **pyTopoComplexity**
### **Landform Smoothing Simulation via Nonlinear Hillslope Diffusion Processes**

This notebook demonstrates the use of **Landlab** (Hobley et al., 2017), an open-source Python framework for simulating landscape evolution, to model topographic smoothing driven by near-surface soil disturbance and downslope soil creep processes. Specifically, it employs the `TaylorNonLinearDiffuser` component from **Landlab**, which is also a key element in the **terrainBento** package (Barnhart et al., 2019), to simulate topographic smoothing over time through nonlinear hillslope diffusion processes (Roering et al., 1999).​

The example GeoTIFF rasters include lidar Digital Elevation Model (DEM) files that cover the area affected by a deep-seated landslide that occurred in 2014 in the Oso area of the North Fork Stillaguamish River valley, Washington State, USA (Washington Geological Survey, 2023). These example DEMs vary in grid spacing (i.e., grid size), coordinate reference system (CRS), and unit of grid value (elevation, Z). One of the goals of this notebook is to enable reproduction of the simulation results presented in Booth et al. (2017).

Archives of codes and example DEM data:
* Zenodo. https://doi.org/10.5281/zenodo.11239338
* Github repository: https://github.com/GeoLarryLai/pyTopoComplexity

<hr>
For more about **Landlab** installation and tutorials, visit: <a href="https://landlab.readthedocs.io/en/latest/index.html">https://landlab.readthedocs.io/en/latest/index.html</a>
<hr>

### **Theories**

##### **Nonlinear Hillslope Diffusion Model**

Inspired by Ganti et al. (2012), the `TaylorNonLinearDiffuser` component employs a slope-dependent flux law with a user-specified number of terms in a Taylor expansion to approximate the Andrews-Bucknam transport function for the nonlinear diffusion process (Roering et al., 1999). The main simulation apply the nonlinear diffusion model iteratively to predict the change of surface elevation $z$ over time $t$. It defines $\mathbf{q}_s$ as a 2D vector representing the rate of soil volume flow per unit slope width (with units of length squared per time, assuming that $\mathbf{q}_s$ represents a ‘bulk’ flux, including pore spaces between soil grains). In the absence of any ‘local’ input sources (such as weathering of rock) or output (such as removal by wash erosion), conservation of mass dictates that

$$\frac{\partial z}{\partial t} = -\nabla \cdot \mathbf{q}_s$$

The soil flux ($\mathbf{q}_s$) can be represented as:

$$\mathbf{q}_s = D \mathbf{S} \left[1 + \sum_{i=1}^N \left( \frac{S}{S_c}\right)^{2i}\right]$$

, where $\mathbf{S} = -\nabla z$ is the downslope topographic gradient, and $S$ is its magnitude. Parameter $D$ is a diffusion-like transport coefficient with dimensions of length squared per time. $S_c$ is the critical slope representing the asymptotic maximum hillslope gradient. $N$ is the number of terms in the Taylor expansion, and the $i$ is the number of additional terms desired. If $N=0$, the expression reduces to plain linear diffusion (Culling, 1963).

##### **Stream Power River Incision Model**

The stream power model accounts detachment-limited fluvial channel erosion processes (assuming negligible sediment effects) in an actively incising landscape. The model predicts the change of topographic elevation $z$ over time $t$ according to the following equation:

$$ \frac{\partial z}{\partial t} = K A^{m} S^{n} $$

Here, $K$ is a positive constant that represents the erodibility coefficient, positively correlated with climate wetness or storminess, and negatively correlated with rock strength. $m$ and $n$ are positive exponents, usually thought to have a ratio, $m/n \approx 0.5$. $A$ is drainage area and $S$ is the slope of steepest descent ($-{\partial z}/{\partial x}$) where $x$ is horizontal distance (positive in the downslope direction) and $z$ is elevation.
<hr>

### **References**
##### Journal Articles: 
* Barnhart, K., Glade, R., Shobe, C., Tucker, G. (2019). Terrainbento 1.0: a Python package for multi-model analysis in long-term drainage basin evolution. Geoscientific Model Development  12(4), 1267-1297. https://doi.org/10.5194/gmd-12-1267-2019
* Booth, A.M., LaHusen, S.R., Duvall, A.R., Montgomery, D.R., 2017. Holocene history of deep-seated landsliding in the North Fork Stillaguamish River valley from surface roughness analysis, radiocarbon dating, and numerical landscape evolution modeling. Journal of Geophysical Research: Earth Surface 122, 456-472. https://doi.org/10.1002/2016JF003934  
* Culling, W.E.H., 1963. Soil creep and the development of hillside slopes. The Journal of Geology 71, 127-161. https://doi.org/10.1086/626891.
* Ganti, V., Passalacqua, P., Foufoula-Georgiou, E. (2012). A sub-grid scale closure for nonlinear hillslope sediment transport models. Journal of Geophysical Research: Earth Surface, 117(F2). https://doi.org/10.1029/2011jf002181
* Hobley, D.E.J., Adams, J.M., Nudurupati, S.S., Hutton, E.W.H., Gasparini, N.M., Istanbulluoglu, E., Tucker, G.E., 2017. Creative computing with Landlab: an open-source toolkit for building, coupling, and exploring two-dimensional numerical models of Earth-surface dynamics. Earth Surf. Dynam. 5, 21-46. https://doi.org/10.5194/esurf-5-21-2017
* Roering, J. J., Kirchner, J. W., & Dietrich, W. E. (1999). Evidence for nonlinear, diffusive sediment transport on hillslopes and implications for landscape morphology. Water Resources Research, 35(3), 853-870. https://doi.org/10.1029/1998wr900090
* Whipple, K. X., & Tucker, G. E. (1999) Dynamics of the stream-power river incision model: Implications for height limits of mountain ranges, landscape response timescales, and research needs. Journal of Geophysical Research, 104(B8), 17661-17694. https://doi.org/10.1029/1999JB900120

##### Digital Elevation Model (DEM) Examples:
* Washington Geological Survey, 2023. 'Stillaguamish 2014' project [lidar data]: originally contracted by Washington State Department of Transportation (WSDOT). [accessed April 4, 2024, at http://lidarportal.dnr.wa.gov]

<hr>

#### 0. Import packages

In [ ]:
import os
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Image
from osgeo import gdal
from landlab import imshowhs_grid
from landlab.components import TaylorNonLinearDiffuser, FlowAccumulator, StreamPowerEroder
from landlab.io import read_esri_ascii, write_esri_ascii

#### 1. Define File Directories
This section defines file directories for input files and simulation results.
The default assumes the input and output GeoTIFF rasters will be placed in the same directory - a subfolder named`ExampleDEM`

In [ ]:
# Input geotiff file and directory
BASE_DIR = os.path.join(os.getcwd(), 'ExampleDEM')
INPUT_TIFF = 'Ososlid2014_f_3ftgrid.tif'

# Setup output directoryresults_
OUT_DIR = os.path.join(BASE_DIR,'simulation_riverhillslope2')
OUT_DIRpng = os.path.join(OUT_DIR, 'PNGs')
OUT_DIRtiff = os.path.join(OUT_DIR, 'GeoTIFFs')
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(OUT_DIRpng, exist_ok=True)
os.makedirs(OUT_DIRtiff, exist_ok=True)

#### 2. Utility Functions
These utility functions handle file format conversions, because **Landlab** are designed for processing raster files in ESRI ASCII format rather than GeoTIFFs.
* `tiff_to_asc`: Converts GeoTIFF to ESRI ASCII format.
* `asc_to_tiff`: Converts ESRI ASCII back to GeoTIFF, preserving metadata.

***Notes***: The input DEM layout must not have nodata or extreme values to prevent Landlab component encountering errors due to infinite slope ($S$) and sediment flux ($\mathbf{q}_s$).

In [ ]:
def tiff_to_asc(in_path, out_path):
    with rasterio.open(in_path) as src:
        XYZunit = src.crs.linear_units   #assuming the unit of XYZ direction are the same
        mean_res = np.mean(src.res)
    
    gdal.Translate(out_path, in_path, format='AAIGrid', xRes=mean_res, yRes=mean_res)
    print(f"The input GeoTIFF is temporarily converted to '{os.path.basename(out_path)}' with grid spacing {mean_res} ({XYZunit})")
    return mean_res, XYZunit
    
def asc_to_tiff(asc_path, tiff_path, meta):
    data = np.loadtxt(asc_path, skiprows=6)
    meta.update(dtype=rasterio.float32, count=1, compress='deflate')

    with rasterio.open(tiff_path, 'w', **meta) as dst:
        dst.write(data.astype(rasterio.float32), 1)
    print(f"'{os.path.basename(tiff_path)}' saved to 'simulation_results_river' folder")

#### 3. Function for Simulation Initialization
This function initializes the simulation:
1. Reads the ASCII file into a **Landlab** grid.
2. Sets boundary conditions.
3. Converts the diffusion coefficient to appropriate units.
4. Creates and returns the output from `TaylorNonLinearDiffuser`,  `StreamPowerEroder`, `FlowAccumulator` components.

In [ ]:
ft2mUS = 1200/3937   #US survey foot to meter conversion factor 
ft2mInt = 0.3048     #International foot to meter conversion factor 

def init_simulation(asc_file, D, K, Sc, XYZunit=None):
    # Read the ASCII file into a Landlab grid
    grid, _ = read_esri_ascii(asc_file, name='topographic__elevation') #the xy grid spacing must be equal
    # Set boundary conditions: open boundaries allow sediment to move in/out of the study area
    grid.set_closed_boundaries_at_grid_edges(False, False, False, False)

    # Check the unit of XYZ and make unit conversion of D and K when needed
    if XYZunit is None:
        print("The function assumes the input XYZ units are in meters.")
    elif XYZunit is not None:
        if any(unit in XYZunit.lower() for unit in ["metre".lower(), "meter".lower()]):
            print("Input XYZ units are in meters. No unit conversion is made")
        elif any(unit in XYZunit.lower() for unit in ["foot".lower(), "feet".lower(), "ft".lower()]):  
            if any(unit in XYZunit.lower() for unit in ["US".lower(), "United States".lower()]):
                print("Input XYZ units are in US survey feet. A unit conversion to meters is made for D and K")
                D = D / (ft2mInt ** 2)    # Convert D from m^2/yr to ft^2/yr
                K = K / (ft2mInt ** 0.5)  # Convert K from m^0.5/yr to ft^0.5/yr
            else:
                print("Input XYZ units are in international feet. A unit conversion to meters is made for D and K")
                D = D / (ft2mInt ** 2)    # Convert D from m^2/yr to ft^2/yr
                K = K / (ft2mInt ** 0.5)  # Convert K from m^0.5/yr to ft^0.5/yr
        else:
            message = (
            "WARNING: The code execution is stopped. "
            "The input XYZ units must be in feet or meters."
            )
            raise RuntimeError(message)
        
    # Initiate Landlab components 
    # For nonlinear hillslope diffusion
    TNLD = TaylorNonLinearDiffuser(grid, linear_diffusivity=D, slope_crit=Sc, dynamic_dt=True, nterms=2, if_unstable = "pass")
    # For flow routing
    fa = FlowAccumulator(grid, flow_director='D8')
    # For stream power erosion
    # K is the erodibility coefficient, m_sp and n_sp are exponents in the stream power equation
    SP = StreamPowerEroder(grid, K_sp=K, m_sp=0.5, n_sp=1.0)
    
    return grid, TNLD, SP, fa


#### 4. Function for Plotting and Saving Results
This function handles visualization and data saving:
1. Creates a hillshade plot of the current topography.
2. Saves the plot as a PNG file.
3. Saves the elevation data as a temporary ASCII file.

In [ ]:
def plot_save(grid, z, basefilename, time, D, K, mean_res=None, XYZunit=None):
    plt.figure(figsize=(6, 5.25))
    imshowhs_grid(grid, z, plot_type="Hillshade")
    plt.title(f"{basefilename} (Nonlinear Diffussion + Stream Power Incision) \n Time: {time} years (D = {D:.1e} m$^{{2}}$/yr, K = {K:.1e} m$^{{0.5}}$/yr)", fontsize='x-small', fontweight="bold")
    plt.xticks(fontsize='small')
    plt.yticks(fontsize='small')
    if XYZunit is not None:
        plt.xlabel(f'X-axis grids \n(grid size ≈ {round(mean_res, 4)} [{XYZunit}])', fontsize='small')
        plt.ylabel(f'Y-axis grids \n(grid size ≈ {round(mean_res, 4)} [{XYZunit}])', fontsize='small')
    else:
        print("The function assumes the input XYZ units are in meters.")
        plt.xlabel(f'X-axis grids \n(grid size ≈ {1.0} [meters])', fontsize = 'small')
        plt.ylabel(f'Y-axis grids \n(grid size ≈ {1.0} [meters])', fontsize = 'small') 
    plt.tight_layout()
    #plt.show()
    plt.savefig(os.path.join(OUT_DIRpng, f"{basefilename}_NLD(D={D:.1e})_SP(K={K:.1e})_{time}yrs.png"), dpi=150)
    plt.close()
    print(f"'{basefilename}_NLD(D={D:.1e})_SP(K={K:.1e})_{time}yrs.png' saved")

    asc_path = os.path.join(OUT_DIRtiff, f"{basefilename}_NLD(D={D:.1e})_SP(K={K:.1e})_{time}yrs.asc")
    write_esri_ascii(asc_path, grid, names=['topographic__elevation'], clobber=True)
    
    return asc_path

#### 5. Main Simulation Function
This is the main function that runs the simulation:
1. Converts input TIFF to ASCII.
2. Initializes the simulation.
3. Runs the diffusion model for the specified number of time steps.
4. For each step, it updates the topography, saves results (Geotiff and PNG), and converts files.
5. Cleans up temporary files (ASCII and PRJ files) at the end.

In [ ]:
def run_simulation(in_tiff, D, K, Sc, dt, target_time):
    basefilename = os.path.splitext(in_tiff)[0]
    in_asc = os.path.join(BASE_DIR, f"{basefilename}.asc")
    mean_res, XYZunit = tiff_to_asc(os.path.join(BASE_DIR, in_tiff), in_asc) #convert input GeoTIFF to ASCII, and determine the XYZ units

    grid, tnld, sp, fa = init_simulation(in_asc, D, K, Sc, XYZunit)
    z = grid.at_node['topographic__elevation']

    with rasterio.open(os.path.join(BASE_DIR, in_tiff)) as src:
        meta = src.meta.copy()

    asc_path = plot_save(grid, z, basefilename, 0, D, K, mean_res, XYZunit)
    tiff_path = os.path.join(OUT_DIRtiff, f"{basefilename}_NLD(D={D:.1e})_SP(K={K:.1e})_0yrs.tif")
    asc_to_tiff(asc_path, tiff_path, meta)
    os.remove(asc_path)

    num_steps = int(target_time / dt)
    for i in range(num_steps):
        small_dt = max(dt / 5, 1)  # Ensure the small timestep is at least 1 year
        for _ in range(int(dt / small_dt)):
            fa.run_one_step()              # Update flow routing
            sp.run_one_step(small_dt)      # Run stream power erosion
            tnld.run_one_step(small_dt)    # Run nonlinear hillslope diffusion
        
        time = (i + 1) * dt
        asc_path = plot_save(grid, z, basefilename, time, D, K, mean_res, XYZunit)
        
        tiff_path = os.path.join(OUT_DIRtiff, f"{basefilename}_NLD(D={D:.1e})_SP(K={K:.1e})_{time}yrs.tif")
        asc_to_tiff(asc_path, tiff_path, meta)
        
        os.remove(asc_path)
        
    in_prj = in_asc.replace('.asc', '.prj')
    os.remove(in_asc)
    os.remove(in_prj)
    print("Simulation completed. Temporary ASCII & PRJ files cleaned up.")

#### 6. Excuting the Simulation
This section sets the simulation parameters and runs the simulation when the script is executed directly. Parameters `Sc`, `D`, `dt`, and `end_time` follow the similar simulation conducted in Booth et al. (2017). Here the critical slope value is set to $S_c = 1.25$ (about $51.34^\circ$), the average slope of the Oso landslide area in the Washington State, USA. The values of the diffusion coefficient $D$ in the cell are adapted from the Figs. 5 & 6 of Booth et al. (2017).

Users can specify the value of `ntems` to determine the number of terms in the Taylor expansion ($N$). The default set two terms in the Taylor expansion (`ntems = 2`) that gives the behavior described in Ganti et al. (2012) as an approximation of the nonlinear diffusion. The code also invokes the `dynamic_dt` option, which allows the component to subdivide each "global" timestep if needed for numerical stability.

In [ ]:
# if __name__ == "__main__":
#     # diffusion coefficient, m^2/y
#     #D = 0.00082
#     #D = 0.0011
#     #D = 0.0015
#     #D = 0.0021
#     D = 0.0029     #Used in Fig.6 of Booth et al. (2017)
#     #D = 0.0041
#     #D = 0.0056
#     #D = 0.0079
#     #D = 0.011
#     #D = 0.016
#     #D = 0.023

#     # Substrate erodibility 'K' in the stream power equation, m^(0.5)/y
#     K = 0.00001

#     Sc = 1.25   # critical slope gradient, m/m
#     dt = 500   # time step size (years)
#     end_time = 15000  # final simulation time (years)

#     run_simulation(INPUT_TIFF, D, K, Sc, dt, end_time)

##### 7. Interactive visualization the simulation results

Users can use a slider to interactively view the simulation results of landscape smoothing at each timestep.

In [ ]:
# def display_image(time):
#     filename = f"{os.path.splitext(INPUT_TIFF)[0]}_NLD(D={D})_SP(K={K})_{time}yrs.png"
#     filepath = os.path.join(OUT_DIRpng, filename)
#     if os.path.exists(filepath):
#         display(Image(filename=filepath))
#     else:
#         print("No file found for this year.")

# print(f"Diffusion coefficient D = {D:.1e} m²/yr")
# print(f"Substrate erodibility K = {K:.1e} m⁰·⁵/yr")

# # Create a slider of time (dt)
# slider = widgets.IntSlider(
#     value=0,
#     min=0,
#     max=end_time,  # end_time should be defined in your earlier cells
#     step=dt,  # dt should be defined as your timestep size in years
#     description='Time (years):', #Name of the slider
#     continuous_update=False
# )

# # Interactive display of the image
# widgets.interactive(display_image, time=slider)

In [ ]:
if __name__ == "__main__":
    # diffusion coefficient, m^2/y
    # D = 0.0029     #Used in Fig.6 of Booth et al. (2017) 
    # D_values = [0.00082, 0.0011, 0.0015, 0.0021, 0.0029, 0.0041, 0.0056, 0.0079, 0.011, 0.016, 0.023]
    D_values = [0.00082, 0.0015, 0.0021, 0.016, 0.023]

    # Substrate erodibility 'K' in the stream power equation, m^(0.5)/y
    # K_values = [0.000001, 0.000005, 0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.00025, 0.000025, 0.000075]
    K = 0.001

    Sc = 1.25   # critical slope gradient, m/m

    # Simulation configurations
    configs = [
        (5, 20),
        (50, 300),
        (500, 15000)
    ]

    for D in D_values:
        for dt, end_time in configs:
            print(f"Running simulation with D={D}, dt={dt}, end_time={end_time}")
            run_simulation(INPUT_TIFF, D, K, Sc, dt, end_time)

In [ ]:
import os
import glob
import imageio
import re

def create_gif(D_value, K_value):
    # Define the input and output directories
    input_dir = OUT_DIRpng
    output_dir = os.path.join(OUT_DIR, 'GIFs')
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Find all PNG files for the given D and K values
    pattern = f"Ososlid2014_f_3ftgrid_NLD(D={D_value:.1e})_SP(K={K_value:.1e})_*.png"
    png_files = glob.glob(os.path.join(input_dir, pattern))
    
    # Sort files based on the year in the filename
    png_files.sort(key=lambda x: int(re.search(r'_(\d+)yrs\.png$', x).group(1)))
    
    # Read all images
    images = [imageio.imread(f) for f in png_files]
    
    # Save the gif
    output_file = os.path.join(output_dir, f'animation_NLD(D={D_value:.1e})_SP(K={K_value:.1e}).gif')
    imageio.mimsave(output_file, images, 'GIF', duration=0.5, loop=2)
    
    print(f"GIF created: {output_file}")

# Get unique D and K values from the existing PNG files
all_files = glob.glob(os.path.join(OUT_DIRpng, 'Ososlid2014_f_3ftgrid_NLD(D=*)_SP(K=*)_*.png'))
D_values = set()
K_values = set()
for f in all_files:
    D_match = re.search(r'NLD\(D=([\d.e-]+)\)', f)
    K_match = re.search(r'SP\(K=([\d.e-]+)\)', f)
    if D_match and K_match:
        D_values.add(float(D_match.group(1)))
        K_values.add(float(K_match.group(1)))

# Create GIF for each D and K value combination
for D in D_values:
    for K in K_values:
        create_gif(D, K)

print("All GIF animations have been created.")
